# Lecture 8 & 9 modern C++

* Part I: ownership and meta-programming
* Part II: more than templates

# Modern C++ part I: ownership and meta-programming

1. Pointers and ownership
2. Revisit shared pointer
3. Template meta-programming and compile-time computing

# Pointer, ownership, smart pointers

* Raw pointer
* Reference
* Ownership
* Smart pointers
  * `unique_ptr`
  * `shared_ptr`

# Modern C++ part II: more than templates

1. Copy elision / RVO, move semantics
2. Variadic template and perfect forwarding
3. Anonymous function, closure, lambda expression